# ONS Convert

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## Imports

Standard python libraries plus determination of projdir, basic printable class, etc

In [1]:
import os
import re
from datetime import datetime, timedelta

import csv
from xlrd import open_workbook, xldate_as_tuple

import numpy as np
from scipy.interpolate import CubicSpline

import common_core
import ons_core
import ons_download

## Constants

Text strings to avoid hard-coded values throughout the code; avoids clutter and silent errors.

In [2]:
# Worksheet names (lower case)
WEEKLY_FIGURES_LOWER = "weekly figures 20"
ESTIMATED_TOTAL_DEATHS_LOWER = "estimated total deaths"
COVID_WEEKLY_REGISTRATIONS_LOWER = "covid-19 - weekly registrations"
COVID_WEEKLY_OCCURRENCES_LOWER = "covid-19 - weekly occurrences"

# Text used to find specific lines
WEEK_NUMBER_TEXT = "Week number"
WEEK_ENDED_TEXT = "Week ended"

# Regular expressions used to find specific lines
TOTAL_DEATHS_REGEX = "^Total deaths, all ages"
TOTAL_OCCURRENCES_REGEX = "^Estimated total death occurrences$"
COVID_DEATHS_REGEX = "^Deaths involving COVID-19, all ages"
RESPIRATORY_REGEX = ".*ICD-10 J00-J99.*"

# Maximum number of columns to search for text / regex
MAX_COLS_WITH_HEADERS = 2

## Facilitate Parsing

Find specific lines in the spreadsheet, etc

In [3]:
def findRowNo(sheet, heading, aliases = {}):
    '''Find rows with the specified headings. Also check for possible aliases.'''

    matches = []

    # Search for row headings with precise wording
    headingLower = heading.lower()

    # Aliases are still regarded as precise wording
    if heading in aliases:
        aliasesLower = [alias.lower() for alias in aliases[heading]]
    else:
        aliasesLower = []

    for rowNo in range(sheet.nrows):
        for colNo in range(MAX_COLS_WITH_HEADERS):
            cellValue = sheet.cell(rowNo, colNo).value

            if isinstance(cellValue, str):
                cellValueLower = cellValue.lower()
                if cellValueLower == headingLower or cellValueLower in aliasesLower:
                    matches.append(rowNo)

    if len(matches) == 0:
        rowNo = -1
    elif len(matches) > 1:
        raise RuntimeError(f"'{heading}' found in '{sheet.name} multiple times - rows {[match + 1 for match in matches]}")
    else:
        rowNo = matches[0]

    return rowNo


def regexFindRowNos(sheet, pattern, verbose = common_core.verbose):
    '''Find rows with the specified headings. Also check for possible aliases.'''

    matches = []

    # Pre-compile regex for minor speedup
    regex = re.compile(pattern)

    for rowNo in range(sheet.nrows):
        for colNo in range(MAX_COLS_WITH_HEADERS):
            cellValue = sheet.cell(rowNo, colNo).value

            if isinstance(cellValue, str):
                if regex.match(cellValue):
                    matches.append(rowNo)

    if len(matches) > 1 and verbose:
        print(f"WARNING: '{pattern}' found in '{sheet.name}' multiple times - rows {[match + 1 for match in matches]}")

    if len(matches) == 0:
        rowNo = -1
    else:
        rowNo = matches[0]

    return rowNo


def getWeekColNos(sheet):
    '''Determine the columns of week numbers from the cells in the specified row.'''

    rowNo = findRowNo(sheet, WEEK_NUMBER_TEXT)
    colNos = []

    for colNo in range(sheet.ncols):
        cellValue = sheet.cell(rowNo, colNo).value

        # If the cell contains a value that can be converted to an integer then treat it as a week number
        try:
            intValue = int(cellValue)
            colNos.append(colNo)
        except:
            pass

    return colNos


def getWeekNumbers(sheet, colNos):
    '''Determine the week numbers from the cells in the specified row.'''

    rowNo = findRowNo(sheet, WEEK_NUMBER_TEXT)
    weekNumbers = []

    for colNo in colNos:
        cellValue = sheet.cell(rowNo, colNo).value

        weekNumbers.append(int(cellValue))

    return weekNumbers


def getWeekEndings(sheet, colNos, dateMode):
    '''Determine the week endings from the cells in the specified row.'''

    rowNo = findRowNo(sheet, WEEK_ENDED_TEXT)
    weekEndings = []

    for colNo in colNos:
        cellValue = sheet.cell(rowNo, colNo).value

        if isinstance(cellValue, str):
            weekEnding = datetime.strptime(cellValue, '%d-%b-%y')
            weekEndings.append(weekEnding.strftime("%Y-%m-%d"))
        else:
            year, month, day, hour, minute, second = xldate_as_tuple(cellValue, dateMode)
            weekEndings.append(f"{year:04}-{month:02}-{day:02}")

    return weekEndings


def getWeekOffsets(weekEndings):
    '''Determine the week endings from the cells in the specified row.'''

    weekOffsets = []

    for weekEnding in weekEndings:
        weekEnding = datetime.strptime(weekEnding, "%Y-%m-%d")
            
        delta = weekEnding.date() - ons_core.minWeek
        weekOffset = delta.days // 7

        weekOffsets.append(weekOffset)

    return weekOffsets


def getCellValue(sheet, rowNo, colNo):
    '''Determine the weekly deaths from the cells in the specified row.'''
    
    cellValue = sheet.cell(rowNo, colNo).value

    # 2011 switched from ICD-10 v 2001 to ICD-10 v 2010 (NCHS)
    # 2014 switched from ICD-10 v 2010 (NCHS) to ICD-10 v 2013 (IRIS)
    if cellValue == ":":
        cellValue = 0

    else:
        # Allow non-integers to be treated as zero but show a warning
        try:
            if cellValue != "":
                cellValue = int(cellValue)
            else:
                cellValue = 0
        except:
            print(f"Warning: Failed to convert '{cellValue}' to integer in '{sheet.name}' (row {rowNo + 1} col {colNo + 1})")
            cellValue = 0

    return cellValue

## Process Spreadsheet

Stuff more specific to the ONS spreadsheets

In [4]:
def initCache(cache, areaNames, verbose = common_core.verbose):
    '''Initialise cache for an individual region'''
    
    dtype = {'names':[ons_core.WEEK_ENDED, ons_core.WEEK_NUMBER,
                      ons_core.TOTAL_REGISTRATIONS, ons_core.TOTAL_OCCURRENCES,
                      ons_core.COVID_REGISTRATIONS, ons_core.COVID_OCCURRENCES],
             'formats':['U10', 'B', 'I', 'I', 'I', 'I']}

    # Calculate the maximum array length
    delta = ons_core.maxWeek - ons_core.minWeek
    maxWeeks = delta.days // 7 + 1

    # Allocate cache for the region
    for areaName in areaNames:
        if areaName not in cache:
            if verbose:
                print(f"Initialising {areaName}...")

            cache[areaName] = np.zeros(maxWeeks, dtype=dtype)

            # Pre-populate week_ended
            for weeksDelta in range(maxWeeks):
                weekEnding = ons_core.minWeek + timedelta(weeks=weeksDelta)
                cache[areaName][ons_core.WEEK_ENDED][weeksDelta] = weekEnding.strftime("%Y-%m-%d")


def processAreas(cache, sheetsInfo, areaNames, verbose = common_core.verbose):
    '''Parse the specified worksheet for weekly deaths in a specific region.'''
    
    initCache(cache, areaNames, verbose = verbose)

    for sheetInfoKey in sheetsInfo:
        sheetInfo = sheetsInfo[sheetInfoKey]

        sheet = sheetInfo["sheet"]
        weekColNos = sheetInfo[ons_core.WEEK_COL_NOS]
        weekNumbers = sheetInfo[ons_core.WEEK_NUMBERS]
        weekEndings = sheetInfo[ons_core.WEEK_ENDINGS]
        weekOffsets = sheetInfo[ons_core.WEEK_OFFSETS]

        for areaName in areaNames:
            if areaName in sheetInfo["regions"]:
                rowNo = sheetInfo["regions"][areaName]

                for i in range(len(weekColNos)):
                    cellValue = getCellValue(sheet, rowNo, weekColNos[i])

                    weekNumber = weekNumbers[i]
                    weekEnding = weekEndings[i]
                    weekOffset = weekOffsets[i]

                    assert cache[areaName][ons_core.WEEK_ENDED][weekOffset] == weekEnding, "Bug in week offset calculations!"

                    cache[areaName][ons_core.WEEK_NUMBER][weekOffset] = weekNumber
                    cache[areaName][sheetInfoKey][weekOffset] = cellValue


def processRegions(cache, sheetsInfo, verbose = common_core.verbose):
    '''Parse the specified worksheet for weekly deaths in a specific region.'''

    regionNames = []
    
    for regionName in common_core.regionNames:
        found = False

        for sheetInfoKey in sheetsInfo:
            sheetInfo = sheetsInfo[sheetInfoKey]
            sheet = sheetInfo["sheet"]

            rowNo = findRowNo(sheet, regionName, common_core.regionAliases)

            if rowNo >= 0:
                sheetInfo["regions"][regionName] = rowNo
                found = True
                
        if found:
            regionNames.append(regionName)

    processAreas(cache, sheetsInfo, regionNames, verbose = verbose)


def processNations(cache, sheetsInfo, verbose = common_core.verbose):
    '''Parse the specified worksheet for weekly deaths in a specific country.'''

    nationNames = []
    
    for nationName in common_core.nationNames:
        found = False

        for sheetInfoKey in sheetsInfo:
            sheetInfo = sheetsInfo[sheetInfoKey]
            sheet = sheetInfo["sheet"]

            if nationName == common_core.ENGLAND_WALES:
                if sheet.name.lower().startswith(WEEKLY_FIGURES_LOWER):
                    rowNo = regexFindRowNos(sheet, TOTAL_DEATHS_REGEX)
                elif sheet.name.lower().startswith(ESTIMATED_TOTAL_DEATHS_LOWER):
                    rowNo = regexFindRowNos(sheet, TOTAL_OCCURRENCES_REGEX)
                else:
                    rowNo = regexFindRowNos(sheet, COVID_DEATHS_REGEX)
            else:
                rowNo = findRowNo(sheet, nationName)

            if rowNo >= 0:
                sheetInfo["regions"][nationName] = rowNo               
                found = True
                
        if found:
            nationNames.append(nationName)
                
    processAreas(cache, sheetsInfo, nationNames, verbose = verbose)


def processSheets(cache, sheetsInfo, dateMode, verbose = common_core.verbose):
    '''Parse the specified worksheets for weekly deaths.'''

    for sheetInfoKey in sheetsInfo:
        sheetInfo = sheetsInfo[sheetInfoKey]
        sheet = sheetInfo["sheet"]

        weekColNos = getWeekColNos(sheet)
        sheetInfo[ons_core.WEEK_COL_NOS] = weekColNos

        weekNos = getWeekNumbers(sheet, weekColNos)
        assert len(weekNos) == len(weekColNos), f"Number of week numbers did not match number of weeks in '{sheet.name}'"
        sheetInfo[ons_core.WEEK_NUMBERS] = weekNos
        
        weekEndings = getWeekEndings(sheet, weekColNos, dateMode)
        assert len(weekEndings) == len(weekColNos), f"Number of week endings did not match number of weeks in '{sheet.name}'"
        sheetInfo[ons_core.WEEK_ENDINGS] = weekEndings

        weekOffsets = getWeekOffsets(weekEndings)
        assert len(weekOffsets) == len(weekEndings), f"Number of week offsets did not match number of weeks in '{sheet.name}'"
        sheetInfo[ons_core.WEEK_OFFSETS] = weekOffsets

        sheetInfo["regions"] = {}

    processNations(cache, sheetsInfo, verbose = verbose)
    processRegions(cache, sheetsInfo, verbose = verbose)


def processWorkbook(cache, workbook, verbose = common_core.verbose):
    '''Parse the specified workbook for weekly deaths.'''

    sheetsInfo = {}

    for sheet in workbook.sheets():           
        if sheet.name.lower().startswith(WEEKLY_FIGURES_LOWER):
            sheetsInfo[ons_core.TOTAL_REGISTRATIONS] = {"sheet": sheet}
        elif sheet.name.lower().startswith(ESTIMATED_TOTAL_DEATHS_LOWER):
            sheetsInfo[ons_core.TOTAL_OCCURRENCES] = {"sheet": sheet}
        elif sheet.name.lower() == COVID_WEEKLY_REGISTRATIONS_LOWER:
            sheetsInfo[ons_core.COVID_REGISTRATIONS] = {"sheet": sheet}
        elif sheet.name.lower() == COVID_WEEKLY_OCCURRENCES_LOWER:
            sheetsInfo[ons_core.COVID_OCCURRENCES] = {"sheet": sheet}
            
    processSheets(cache, sheetsInfo, workbook.datemode, verbose = verbose)


def getWorkbookYear(workbook):
    '''Scan the specified workbook to determine the year.'''

    year = None

    for sheet in workbook.sheets():           
        if sheet.name.lower().startswith(WEEKLY_FIGURES_LOWER):
            year = int(sheet.name[-4:])

    if year == None:
        raise RuntimeError(f"Year could not be determined for workbook")

    return year

## Spreadsheet Interface

Main interface for converting from XLSX files to CSV files

In [5]:
def loadExcelFiles(partNames, verbose = common_core.verbose):
    '''Load the specified spreadsheets into cache.'''

    # Iterate through all workbooks to determine the years
    years = {}
    for partName in partNames:
        fileName = os.path.join(common_core.projdir, "data", "ons-deaths", "raw", partName)
        workbook = open_workbook(fileName)

        year = getWorkbookYear(workbook)
        years[year] = workbook

    # Iterate throught the years in chronological order - required to handle the 2021 hybrid!
    cache = {}
    for year in sorted(years):
        try:
            processWorkbook(cache, years[year], verbose = verbose)
        except:
            print(f"ERROR: Exception raise whilst processing workbook for {year}")
            raise

    return(cache)

## Cache Enhancement

Enhance data in the cache

In [6]:
def createEngland(cache, verbose = common_core.verbose):
    '''Create data for England using England and Wales'''

    initCache(cache, [common_core.ENGLAND], verbose = verbose)
    
    for regionName in common_core.regionNames:
        cache[common_core.ENGLAND][ons_core.TOTAL_REGISTRATIONS] += cache[regionName][ons_core.TOTAL_REGISTRATIONS]
        cache[common_core.ENGLAND][ons_core.TOTAL_OCCURRENCES] += cache[regionName][ons_core.TOTAL_OCCURRENCES]

        cache[common_core.ENGLAND][ons_core.COVID_REGISTRATIONS] += cache[regionName][ons_core.COVID_REGISTRATIONS]
        cache[common_core.ENGLAND][ons_core.COVID_OCCURRENCES] += cache[regionName][ons_core.COVID_OCCURRENCES]

In [7]:
def trimCache(cache, verbose = common_core.verbose):
    '''Remove rows from cache which don't contain any useful data'''

    for areaName in cache:
        populated = np.where(cache[areaName][ons_core.TOTAL_REGISTRATIONS] > 0)[0]
        cache[areaName] = cache[areaName][populated[0]:populated[-1] + 1]

In [8]:
def calculateOccurrencesW53(cache, verbose = common_core.verbose):
    '''Calculate the estimated number of occurrences for week 53 of 2020'''

    indexW53 = np.where(cache[common_core.ENGLAND_WALES][ons_core.WEEK_ENDED] == '2021-01-01')[0][0]
    
    # Take minimum of 4 weeks before and 4 weeks afterwards
    yBefore = cache[common_core.ENGLAND_WALES][ons_core.TOTAL_OCCURRENCES][indexW53 - 4:indexW53]
    yAfter = cache[common_core.ENGLAND_WALES][ons_core.TOTAL_OCCURRENCES][indexW53 + 1:indexW53 + 5]
    y = np.hstack((yBefore, yAfter))
    
    # y values need to correspond to the x values
    xBefore = np.arange(len(yBefore))
    xAfter = np.arange(len(yAfter)) + 5
    x = np.hstack((xBefore, xAfter))
    
    # Calculate the missing point using Cubic Spline
    cs = CubicSpline(x, y)

    cache[common_core.ENGLAND_WALES][ons_core.TOTAL_OCCURRENCES][indexW53] = cs(4)
    

def polyfillCache(cache, verbose = common_core.verbose):
    '''Calculate missing values using whatever method is appropriate'''

    calculateOccurrencesW53(cache, verbose = verbose)

## Save Cache Data

Save data in numpy arrays to CSV files

In [9]:
def saveArea(cache, areaType, areaName, verbose = common_core.verbose):
    '''Save data in cache to CSV'''

    if areaName in cache:
        if verbose:
            print(f"Saving {areaName}...")

        header = ','.join(cache[areaName].dtype.names)

        # Ensure CSV path exists
        csvPath = os.path.join(common_core.projdir, "data", "ons-deaths", "csv", "weekly", "deaths", areaType)
        if not os.path.exists(csvPath):
            os.makedirs(csvPath)

        # Determine safe filename
        csvFn = os.path.join(csvPath, common_core.getSafeName(areaName) + ".csv")

        # Save data to CSV
        np.savetxt(csvFn, cache[areaName], fmt='%s', delimiter=',', header=header, comments='')


def saveCache(cache, verbose = common_core.verbose):
    '''Save all extracted data to CSV'''

    for nationName in common_core.nationNames:
        saveArea(cache, "nation", nationName, verbose = verbose)
        
    for regionName in common_core.regionNames:
        saveArea(cache, "region", regionName, verbose = verbose)

## Interactive Testing

In [10]:
if __name__ == '__main__':

    verbose = True

    # Check / download latest spreadsheets
    partNames = ons_download.downloadDeaths(verbose = verbose)
    cache = loadExcelFiles(partNames, verbose = verbose)

    # Tidy up the data, prior to saving as CSV files
    createEngland(cache, verbose = verbose)
    trimCache(cache, verbose = verbose)
    polyfillCache(cache, verbose = verbose)

    # Save cache as CSV files
    saveCache(cache, verbose = verbose)

Skipping download of weekly/publishedweek012021.xlsx...
Skipping download of weekly/publishedweek532020.xlsx...
Skipping download of weekly/publishedweek522019.xls...
Skipping download of weekly/publishedweek522018withupdatedrespiratoryrow.xls...
Skipping download of weekly/publishedweek522017.xls...
Skipping download of weekly/publishedweek522016.xls...
Skipping download of weekly/publishedweek2015.xls...
Skipping download of weekly/publishedweek2014.xls...
Skipping download of weekly/publishedweek2013.xls...
Skipping download of weekly/publishedweek2012.xls...
Skipping download of weekly/publishedweek2011.xls...
Skipping download of weekly/publishedweek2010.xls...
Initialising England and Wales...
Initialising Wales...
Initialising North East...
Initialising North West...
Initialising Yorkshire and The Humber...
Initialising East Midlands...
Initialising West Midlands...
Initialising East of England...
Initialising London...
Initialising South East...
Initialising South West...
Initi